In [ ]:
%load_ext autoreload
%autoreload 2

import dotdict as dd
from dotdict import DotDict

In [ ]:
a = {'a': 10, 'b': 71}
aa = DotDict(a)
aa

In [ ]:
b = {'a': 4, 'b': {'c': 100, 'd': 10239}}
bb = DotDict(b, debug=False)
bb

In [ ]:
c = DotDict(one=1, two=2)
c

In [ ]:
print(
    bb['a'],
    bb.a,
    bb['b'],
    bb.b,
    bb.debug,
    sep='\n'
)

In [ ]:
bb.debug = True

In [ ]:
bb.debug

In [ ]:
bb.toggle_debug()

In [ ]:
dict(bb)
# Casting back to a dict doesn't automatically convert all inner
# dotdicts to dicts.

In [ ]:
bb.as_dict()

In [ ]:
bb.__dict__

In [ ]:
bb.c = 10

In [ ]:
bb._hoi = 5

In [ ]:
bb._DotDict__hoi = 10

Partition implementations speed comparison

In [ ]:
q = list(range(1000))
r = lambda n: n % 2

In [ ]:
%timeit dd.partition(r, q)

In [ ]:
%timeit dd.partitionx(r, q)

Speed comparison between `filter()` in `__iter__` and `as_dict()`

In [ ]:
from timing import time_func
time_func(dict, [bb], 1000000)

`as_dict()`
```
Total time taken: 0.766301978030242 
Average time per iteration: 7.66301978030242e-07 
Standard deviation: 2.9076185924853917e-07
Median: 7.500057108700275e-07 
Longest time: 0.00010466598905622959 
Shortest time: 6.249756552278996e-07
```

Updated `as_dict()`
```
Total time taken: 0.7289091332349926 
Average time per iteration: 7.289091332349926e-07 
Standard deviation: 2.2147264020848343e-07
Median: 7.080088835209608e-07 
Longest time: 8.95409903023392e-05 
Shortest time: 5.829788278788328e-07
```

`filter()`
```
Total time taken: 0.9377548526390456 
Average time per iteration: 9.377548526390456e-07 
Standard deviation: 2.592479778803312e-07
Median: 9.170034900307655e-07 
Longest time: 0.00011566700413823128 
Shortest time: 7.909839041531086e-07
```

I guess what I expected, since filter will move through all items and a dictionary `.pop()` method will likely use some kind of binary search. This means the complexity of `filter()` will be O(n) and that of `.pop()` is O(log n).

Interesting idea nonetheless so I will save the code below.

In [ ]:
def __iter__(self):
        def __filter(item):
            key, _ = item
            return key != self.__prefix + '__settings'

        return filter(__filter, self.__dict__.items())

Maybe important note:

Note that `super()` is implemented as part of the binding process for explicit dotted attribute lookups such as `super().__getitem__(name)`. It does so by implementing its own `__getattribute__()` method for searching classes in a predictable order that supports cooperative multiple inheritance. Accordingly, `super()` is undefined for implicit lookups using statements or operators such as `super()[name]`.

From: https://docs.python.org/3/library/functions.html#super

But I guess it also means my intuition about using `super()__getattribute__(__name)` for accessing settings is right. (even though it might internally be hacky.)

Important note: (from https://wiki.python.org/moin/UsingSlots)

By default, when Python creates a new instance of a class, it creates a `__dict__` attribute for the class. The `__dict__` attribute is a dictionary whose keys are the variable names and whose values are the variable values. This allows for dynamic variable creation but can also lead to uncaught errors.

So if you want to implement `__setitem__` and `__getitem__`, is it not better to directly edit the `__dict__`? Instead of what you're doing now: call `self.__setattr__()`

Testing which is faster:
```py
__data = dict(__data, **kwargs)
```
or:
```py
if kwargs:
    __data = dict(__data, **kwargs)
```

In [ ]:
params = [a]
kwargs = {'one': 1}

In [ ]:
time_func(DotDict, params, iterations=1000000, kwargs=kwargs)

Saved as markdown for future reference:

Old (without if):
```
Total time taken: 1.2708917100826511 
Average time per iteration: 1.2708917100826511e-06 
Standard deviation: 4.773030086862648e-07
Median: 1.2499949662014842e-06 
Longest time: 9.908300125971437e-05 
Shortest time: 1.0829971870407462e-06
```

New (with if):
```
Total time taken: 1.194327569566667 
Average time per iteration: 1.194327569566667e-06 
Standard deviation: 3.6330861553044284e-07
Median: 1.167005393654108e-06 
Longest time: 9.662499360274523e-05 
Shortest time: 1.0410003596916795e-06
```

So new is definitely better. With the if-statement!